In [ ]:
#coding: utf-8
#BUGS:
'''
due to matplotlib's different behavior on ios and windows, the
course change graph does not display will all info, since the
first window must be closed in order for the new info to appear, 
and that takes a new window
'''
# 



import math
import matplotlib.pyplot as plt
import numpy as np

def cart2pol(x, y):
    rho = np.sqrt(x**2 + y**2)
    phi = np.arctan2(x,y) #inversed due to negative plot direction
    return(rho, phi)

def pol2cart(rho, phi):# since direction of plot is neg, these vals are reversed
    ydelta = rho * np.cos(phi)
    xdelta = rho * np.sin(phi)
    return(xdelta, ydelta)
    
def get_speed(p1,p2):
    xdelta = p1[0] - p2[0]
    ydelta = p1[1] - p2[1]
    spd = np.math.sqrt(xdelta*xdelta + ydelta*ydelta)
    return np.round(spd * 10)

def get_course(p1,p2):
    xdelta = p1[0] - p2[0]
    ydelta = p1[1] - p2[1]
    r,f = cart2pol(xdelta,ydelta)
    return np.round(math.degrees(f) % 360)

def vector_target_cartesian_coord(p,c,v,vl=6,contra=True):#sourcePoint, course, speed
    if contra:
        course = (c - 180) % 360
    else:
        course = c % 360

    ydelta= np.math.cos(np.math.radians(course)) * v / (60./vl)
    xdelta = np.math.sin(np.math.radians(course)) * v / (60./vl)

    x = p[0] + xdelta
    y = p[1] + ydelta	

    return x,y

def vector_slope(o,a):
    #BUG: any vertical line results in dev by zero! 
    # called to calculate RML and NRML
    namnare = o[0] - a[0]
    slope = (o[1] - a[1]) / namnare # k = y2-y1/x2-x1
    return slope

def vector_m(o,slope):
    #o[1] = slope * o[0] + m => m = o[1] - (slope * o[0])
    return o[1] - (slope * o[0])

def vector_y_0(k,m):
    x = -m / k
    return x

def calc_y(x,slope,m):
    #global slope,m
    y = slope*x+m
    return y

def closest_to_origo(x,y):
    h = np.array(x**2+y**2)
    h = np.sqrt(h)
    min_idx = h.argmin()
    cpa = h.min()
    return (x[min_idx],y[min_idx],cpa)

def dist_p_p(p1,p2):
    xdelta = p1[0] - p2[0]
    ydelta = p1[1] - p2[1]
    h = math.sqrt(xdelta**2 + ydelta**2)
    return h
    
def draw_vector(p1,p2,ax,m,c,ls='-'): #call must be after ax is defined
    polar_p1 = cart2pol(p1[0],p1[1])
    polar_p2 = cart2pol(p2[0],p2[1])
    theta = np.array([polar_p1[1],polar_p2[1]])
    r = np.array([polar_p1[0],polar_p2[0]])
    ax.plot(theta,r,marker=m,color=c,linewidth=1)

def get_data(test=False):
    if not test:
        try:
            co = float(input('Own Course [000]:'))
        except:
            co = 0
        try:
            vo = float(input('Own Speed [20 kn]:'))
        except:
            vo = 20
        try:
            vl = float(input('vector length [6 min]:'))
        except:
            vl = 6
        b1 = float(input('Bearing 1:'))
        r1 = float(input('Range 1:'))
        b2 = float(input('Bearing 2:'))
        r2 = float(input('Range 2:'))
    else:
        co = 0
        vo = 20
        vl = 6
        b1 = 45
        r1 = 8
        b2 = 44
        r2 = 6
    return co,vo,b1,r1,b2,r2,vl

def offset_circle(offset,r):
    x_offset,y_offset = (offset[0],offset[1]) #centre of circle
    
    degrees = np.array(np.arange(370,step=10))
    xs = np.sin(np.radians(degrees)) * r #reversed due to negative plot direction
    ys = np.cos(np.radians(degrees)) * r  #reversed due to negative plot direction
    polars = cart2pol(xs+x_offset,ys+y_offset)
    polar_arr = np.array(list(zip(polars[0],polars[1])))
    theta = polar_arr[:,1]
    rho = polar_arr[:,0] 
    
    return rho,theta
    
def nc_polar_vector(offset,degrees,r):
    x = np.sin(np.radians(degrees)) * r
    y = np.cos(np.radians(degrees)) * r
    polars = cart2pol(x+offset[0],y+offset[1])
    return polars
    
def cpa_point(slope,m,p1,p2):
    cpax = np.array(np.linspace(-rmax,rmax,1000))#numeric method for finding cpa
    cpay = calc_y(cpax,slope,m)
    cpa_p = closest_to_origo(cpax,cpay)
    cpa = cpa_p[2]
    dist_p1_p2 = dist_p_p(p1,p2)
    dist_p2_cpa = dist_p_p((cpa_p[0],cpa_p[1]),p2)
    rel_spd = dist_p1_p2 * 10
    if rel_spd == 0:
        print ('Medåkare!')
        rel_spd = 0.0000000001
    tcpa = np.round(dist_p2_cpa / rel_spd * 60,1)
    polar_cpa_p = cart2pol(cpa_p[0],cpa_p[1])
    cpa_theta = np.array([0,polar_cpa_p[1]])
    cpa_r = np.array([0,polar_cpa_p[0]])
    return cpa_theta,cpa_r,cpa_p,cpa,tcpa,polar_cpa_p,rel_spd

def cpa_point_000_180(p1,p2,medakare):
    
    cpa_p = p1[0],0,p1[0]
    cpa = cpa_p[2]
    dist_p1_p2 = dist_p_p(p1,p2)
    dist_p2_cpa = dist_p_p((cpa_p[0],cpa_p[1]),p2)
    rel_spd = dist_p1_p2 *10
    if rel_spd > 0.0:
        tcpa = np.round(dist_p2_cpa / rel_spd * 60,1)
    else:
            tcpa = 999999999
            medakare = True
    polar_cpa_p = cart2pol(cpa_p[0],cpa_p[1])
    cpa_theta = np.array([0,polar_cpa_p[1]])
    cpa_r = np.array([0,polar_cpa_p[0]])
    return cpa_theta,cpa_r,cpa_p,cpa,tcpa,polar_cpa_p,rel_spd,medakare
    
#### main ####
test = False

rmax = 12. #plot size
medakare = False


ui = get_data(test)

b = [ui[2],ui[4]] #plotted bearings
r = [ui[3],ui[5]] #plotted ranges
vo = ui[1] #kn #own speed
co = ui[0]#own course
vl = ui[6] 


####        ####

#plotted coordinates
(x1,y1) = pol2cart(r[0],math.radians(b[0])) #point o
(x3,y3) = pol2cart(r[1],math.radians(b[1])) #point a

o = (x1,y1)
wx,wy = vector_target_cartesian_coord(o,co,vo) #x,y for point w
polarw = cart2pol(wx,wy)#polar coord for w
wtheta = np.array([math.radians(b[0]),polarw[1]]) #angle vector o->w
wr = np.array([r[0],polarw[0]]) #radius vector o->w

polara = cart2pol(x3,y3)# polars for point a... unnecessary...? 
atheta = np.array([polarw[1],polara[1]])# angle vector w->a
ar = np.array([polarw[0],polara[0]])#radius vector w->a

# spd and course target
vt = get_speed((wx,wy),(x3,y3))
ct = get_course((x3,y3),(wx,wy))

#RML extended line

if np.round(x1,5) == np.round(x3,5): #vertical RML,cant use line eq.
    cpa_theta,cpa_r,cpa_p,cpa,tcpa,polar_cpa_p,rel_spd,medakare = cpa_point_000_180((x1,y1),(x3,y3),medakare)
    
else:
    rml_slope = vector_slope((x1,y1),(x3,y3))
    rml_m = vector_m((x1,y1),rml_slope)
    cpa_theta,cpa_r,cpa_p,cpa,tcpa,polar_cpa_p,rel_spd= cpa_point(rml_slope,rml_m,(x1,y1),(x3,y3))

c_RML = get_course(cpa_p,(x3,y3))
rv_end_point = vector_target_cartesian_coord((x3,y3),c_RML,rel_spd,vl,False)
polar_rv_end_point = cart2pol(rv_end_point[0],rv_end_point[1])
rv_theta = np.array([polara[1],polar_rv_end_point[1]])
rv_r = np.array([polara[0],polar_rv_end_point[0]])

rml_ext_theta = np.array([polara[1],polar_cpa_p[1]])
rml_ext_r = np.array([polara[0],polar_cpa_p[0]])

#True Vector
tv_end_point = vector_target_cartesian_coord((x3,y3),ct,vt,vl,False)
polar_tv_end_point = cart2pol(tv_end_point[0],tv_end_point[1])
tv_theta = np.array([polara[1],polar_tv_end_point[1]])
tv_r = np.array([polara[0],polar_tv_end_point[0]])


print ('Own Course:',co,'Own Speed:',vo,'Vector Length:',vl)
print ('Target Plots R/B:',zip(r,b))
print ('Target Speed:',vt, 'Target Course:',ct)

if not medakare:
    print ('CPA:',np.round(cpa,2),'TCPA:',tcpa,'min')
else:
    print ('Medåkare, CPA:',r[0],'TCPA: N/A')

# own origo based vector
own_vector_theta = np.array([math.radians(co),math.radians(co)])
own_vector_rho = np.array([0,vo/(60./vl)])

#plot relative track
rml_theta = np.array([math.radians(b[0]),math.radians(b[1])])
rml_rr = np.array(r)

#offset circle 
circle_r,circle_theta = offset_circle((wx,wy),vo/10.)

fig = plt.figure(figsize=(18,12))
ax = plt.subplot(111, polar=True)

ax.plot(rml_theta, rml_rr, marker='o',color='r', linewidth=5)#RML,o->a

if not medakare:
    ax.plot(atheta,ar,marker='+',color='g',linewidth=5)#w->a
    ax.plot(rml_ext_theta,rml_ext_r,marker='+',color='r',linewidth=1) #RML extension,a->cpa
    ax.plot(cpa_theta, cpa_r, marker='+',color='c',linewidth=1) #Origo -> cpa
    ax.plot(wtheta,wr,marker='x',color='b',linewidth=5)#o->w
    
ax.plot(tv_theta,tv_r,marker='*',color='g',linewidth=1)

ax.plot(rv_theta,rv_r,marker='*',color='r') #target RV
ax.plot(circle_theta,circle_r,color='y',linewidth=1) 

ax.plot(own_vector_theta,own_vector_rho,marker='*',color='b',linewidth=1)


ax.set_rmax(rmax)
ax.grid(True)
ax.set_theta_zero_location("N")
ax.set_theta_direction(-1)
ax.set_title("NU Radar Plot", va='bottom')

try:
    msg = 'New Course [' + str(co) + ']:'
    nc = int(input(msg))
except:
    nc = co
try:
    msg = 'New Speed [' + str(vo) + ']:'
    nv = int(input(msg))
except:
    nv = vo

# converging or diverging courses
# tbd

#plot own new vector
nc_own_vector_theta = np.array([math.radians(int(nc)),math.radians(int(nc))])

nc_own_vector_rho = np.array([0,nv/(60./vl)])
ax.plot(nc_own_vector_theta,nc_own_vector_rho,marker='*',color='b')

#new course vector i plotting triangle
polar_nc=nc_polar_vector((wx,wy),nc,nv/10.) 
cart_nc = pol2cart(polar_nc[0],polar_nc[1])
nc_polar_vector_theta = np.array([polarw[1],polar_nc[1]])
nc_polar_vector_r = np.array([polarw[0],polar_nc[0]])

#New relative vector
nc_nrml_slope = vector_slope(cart_nc,(x3,y3))

nc_nrml_m = vector_m(cart_nc,nc_nrml_slope)

nc_cpa_theta,nc_cpa_r,nc_cpa_p,nc_cpa,nc_tcpa,nc_polar,nc_rel_spd = cpa_point(nc_nrml_slope,nc_nrml_m,cart_nc,(x3,y3))

nc_RML = get_course(nc_cpa_p,(x3,y3))

nc_rv_end_point = vector_target_cartesian_coord((x3,y3),nc_RML,nc_rel_spd,vl,medakare)

nc_polar_rv_end_point = cart2pol(nc_rv_end_point[0],nc_rv_end_point[1])
nc_rv_theta = np.array([polara[1],nc_polar_rv_end_point[1]])
nc_rv_r = np.array([polara[0],nc_polar_rv_end_point[0]])

draw_vector(cart_nc, (x3,y3),ax,'*','k') #nc_RML

# TV pencils
nc_w_point = vector_target_cartesian_coord(tv_end_point,nc,nv,vl,True) 
draw_vector(tv_end_point,nc_w_point,ax,'*','b')
draw_vector((x3,y3),nc_w_point,ax,'*','k')

diverging = False

if not medakare:
    draw_vector((x3,y3),nc_cpa_p, ax,'*','k')#nc_RML-extention
    print ('New CPA:',np.round(nc_cpa,1),'New TCPA:',nc_tcpa,'min')
else: 
    if cart2pol(nc_w_point[0],nc_w_point[1])[0] > cart2pol(x3,y3)[0]:
        diverging = True
        print ('Diverging courses, CPA:',r[0],'TCPA:N/A')
    else:
        print ('CPA:', np.round(nc_cpa,1),'New TCPA:',nc_tcpa,'min')
        draw_vector((x3,y3),nc_cpa_p, ax,'*','k')#nc_RML-extention

ax2 = plt.subplot(111,polar=True)
ax2.plot(nc_polar_vector_theta,nc_polar_vector_r,marker='*',color='b',linewidth=3)# new course in plot triangle

if not diverging:
    ax2.plot(nc_cpa_theta,nc_cpa_r,marker='+',color='c',linewidth=1,linestyle='-') # new cpa line
    
ax2.plot(nc_rv_theta, nc_rv_r,marker='*',color='k')
ax2.set_rmax(rmax)
ax2.grid(True)
ax2.set_theta_zero_location("N")
ax2.set_theta_direction(-1)
ax2.set_title("NU Radar Plot", va='bottom')